### Rules:
- 18 weeks total
- Each team has one bye week from week 4-14
- Each team plays 17 games
    - 6 divisional games: play each divisional rival 2 times, 
    - 1 game is against a team from the other conference
    - the other 10 are random teams in the same conference

In [294]:
import numpy as np
import copy, json, random, time

In [317]:
class NFLTeam():
    def __init__(self, city, mascot, conference, division):
        self.name = city + " " + mascot
        self.city = city
        self.mascot = mascot

        self.conference = conference
        self.division = division

        self.conference_games_played = 0
        self.division_games_played = 0

        self.schedule = {i: None for i in range(1,19)}
        self.schedule_outline = {i: None for i in range(1,19)}
        self.bye_week = None

        self.required_opponents = {"division": None, "conference": None}
        self.opponents_not_faced = []

    def __repr__(self):
        return f"{self.mascot} {self.conference[0]}.{self.division[0].lower()}"

    def __str__(self):
        # return f"{self.name} ({self.conference} {self.division}) - Bye Week: {self.bye_week} - Games Played: {self.games_played}"
        return self.__repr__()
    
    def set_bye_week(self, week_num):
        """assigns week_num (int) as the bye week"""
        self.bye_week = week_num
        self.schedule_outline[week_num] = "-- BYE --"
        self.schedule[self.bye_week] = " -- BYE --"

    def play_game(self, opponent, week_number, home_indicator):
        self.games_played += 1

        if opponent.conference == self.conference:
            self.conference_games_played += 1
            if opponent.division == self.division:
                self.division_games_played += 1

        # delete the opponent from the list of other teams that this team has not played against
        if opponent in self.opponents_not_faced:
            self.opponents_not_faced.remove(opponent)

        # add this game to the team objects schedule
        self.schedule.append({
            'week': week_number,
            'opponent': opponent,
            'home_ind': home_indicator  # You can determine if it's a home game based on your scheduling logic
        })

    def add_game(self, week, opponent):
        self.schedule[int(week)] = opponent

In [319]:
def create_teams() -> list:
    """ Returns a list of the 32 NFLTeam objects, one for each team"""

    # AFC East Teams
    patriots = NFLTeam("New England", "Patriots", "AFC", "East")
    bills = NFLTeam("Buffalo", "Bills", "AFC", "East")
    dolphins = NFLTeam("Miami", "Dolphins", "AFC", "East")
    jets = NFLTeam("New York", "Jets", "AFC", "East")

    # AFC North Teams
    ravens = NFLTeam("Baltimore", "Ravens", "AFC", "North")
    steelers = NFLTeam("Pittsburgh", "Steelers", "AFC", "North")
    browns = NFLTeam("Cleveland", "Browns", "AFC", "North")
    bengals = NFLTeam("Cincinnati", "Bengals", "AFC", "North")

    # AFC South Teams
    texans = NFLTeam("Houston", "Texans", "AFC", "South")
    colts = NFLTeam("Indianapolis", "Colts", "AFC", "South")
    titans = NFLTeam("Tennessee", "Titans", "AFC", "South")
    jaguars = NFLTeam("Jacksonville", "Jaguars", "AFC", "South")

    # AFC West Teams
    chiefs = NFLTeam("Kansas City", "Chiefs", "AFC", "West")
    broncos = NFLTeam("Denver", "Broncos", "AFC", "West")
    raiders = NFLTeam("Las Vegas", "Raiders", "AFC", "West")
    chargers = NFLTeam("Los Angeles", "Chargers", "AFC", "West")

    # NFC East Teams
    cowboys = NFLTeam("Dallas", "Cowboys", "NFC", "East")
    washington = NFLTeam("Washington", "Commanders", "NFC", "East")
    eagles = NFLTeam("Philadelphia", "Eagles", "NFC", "East")
    giants = NFLTeam("New York", "Giants", "NFC", "East")

    # NFC North Teams
    packers = NFLTeam("Green Bay", "Packers", "NFC", "North")
    bears = NFLTeam("Chicago", "Bears", "NFC", "North")
    vikings = NFLTeam("Minnesota", "Vikings", "NFC", "North")
    lions = NFLTeam("Detroit", "Lions", "NFC", "North")

    # NFC South Teams
    buccaneers = NFLTeam("Tampa Bay", "Buccaneers", "NFC", "South")
    saints = NFLTeam("New Orleans", "Saints", "NFC", "South")
    panthers = NFLTeam("Carolina", "Panthers", "NFC", "South")
    falcons = NFLTeam("Atlanta", "Falcons", "NFC", "South")

    # NFC West Teams
    seahawks = NFLTeam("Seattle", "Seahawks", "NFC", "West")
    rams = NFLTeam("Los Angeles", "Rams", "NFC", "West")
    cardinals = NFLTeam("Arizon", "Cardinals", "NFC", "West")
    sf49ers = NFLTeam("San Francisco", "49ers", "NFC", "West")

    # Create a list of all 32 NFLTeam objects
    return [patriots, bills, dolphins, jets, ravens, steelers, browns, bengals,
            texans, colts, titans, jaguars,chiefs, broncos, raiders, chargers,
            cowboys, washington, eagles, giants,packers, bears, vikings, lions,
            buccaneers, saints, panthers, falcons, seahawks, rams, cardinals, sf49ers]

In [340]:
class NFLSchedule():
    """
    Holds the master schedule for the NFL Season, optional parameter weeks set to default = 18
    """
    def __init__(self, all_NFL_teams, weeks = 18):
        self.allteams = all_NFL_teams
        self.AFC = [ team for team in self.allteams if "AFC" in team.conference.upper() ]
        self.NFC = [ team for team in self.allteams if "NFC" in team.conference.upper() ]
        self.teams_played_other_conference = []
        # 
        self.schedule = {i: None for i in range(1, weeks+1)}

    def __str__(self):
        # return json.dumps(self.schedule, indent=4)
        return self.allteams

    def weekly_bye_count(self, eligible_weeks):
        # keep randomly creating the number of teeams on bye for each week until there are 32 bye slots
        weekly_bye_count_list = [random.choice([2,4]) for week in eligible_weeks]
        while sum(weekly_bye_count_list) != 32:
            weekly_bye_count_list =  [random.choice([2,4]) for week in eligible_weeks]

        # add the weeks at start and end of season to fill out the full schedule
        weekly_bye_count_list = [0,0,0] + weekly_bye_count_list + [0,0,0,0]

        # list of how many teams from each conference are on a bye for in each week (index)
        weekly_bye_slots_per_conference = [value//2 for value in weekly_bye_count_list]
        return weekly_bye_slots_per_conference

    def assign_bye_weeks(self, debug = False):
        """Creates the bye weeks and assign each team's .bye_week attribute
            - This works most of the time, but still fails ~15% of the time. it either works in the first 10 attempts or it fails 10,000 attempts. 
                - Must have something to do with the 
            
            Bye Week Rules:
            - byes occur from week 4 through weeek 14
            - either 2 or 4 teams are on a bye in any given week
            - must choose same number of teams from the AFC and NFC each week
            - each team has 1 bye week per season

        Args:
            league (list): a list of all NFLTeam objects in the league

        Returns:
            *** NOTE: the returned dict is just a visual aid ***
            (dict): a dictionary with key = week_num (int) and value = list of [team.name, team.conference] for each team on bye that week 
        """
        
        # Byes can only occur from week 4 to week 14 (i.e. range index 3 - 13)
        eligible_weeks = list(range(4, 15)) 
        weekly_bye_slots_per_conference = self.weekly_bye_count(eligible_weeks)
        if debug:
            print(f"{weekly_bye_slots_per_conference = }")

        # Set up a while loop so that if it tries to assign a bye week on a divisional week, it will start over and try again.
        keep_searching = True        
        count = 0
        if debug:
            cutoff = 1
        else:
            cutoff = 10000

        while keep_searching == True:
            random.seed(time.time())

            count += 1
            if debug:
                print(f"{count = }")
            if count > cutoff:
                break

            # create the empty bye list to keep track of who has been assigned a bye week already
            bye_list = {team: None for team in self.allteams}

            # create temp copies of the self attributes that we need for this round
            AFC_teams = self.AFC.copy()
            NFC_teams = self.NFC.copy()

            # AFC_teams = list(np.random.shuffle(AFC_teams))
            # NFC_teams = list(np.random.shuffle(NFC_teams))

            # for each week when byes can occur, choose "num_slots" teams who are not in divisional play this week for a bye
            for week_num in eligible_weeks:
                if debug:
                    print("_"*50, f"\n{week_num = }")
                for num_slots in range(weekly_bye_slots_per_conference[week_num-1]):
                    # grab 1 or 2 teams from each conference, depending on how many slots are available 
                    eligible_teams_afc = [team for team in AFC_teams if team.schedule_outline[week_num] == None]
                    eligible_teams_nfc = [team for team in NFC_teams if team.schedule_outline[week_num] == None]

                    if debug:
                        print()
                        print(f"{weekly_bye_slots_per_conference[week_num] = }")
                        print(f"{AFC_teams = }")
                        print(f"{NFC_teams = }")
                        print(f"{eligible_teams_afc = }")
                        print(f"{eligible_teams_nfc = }")
                        print()

                    if eligible_teams_afc and eligible_teams_nfc:
                    # if eligible_teams_afc.any() and eligible_teams_nfc.any():
                        """choose AFC team(s)"""
                        selected_team_a = random.choice(eligible_teams_afc)
                        # selected_team_a = eligible_teams_afc[0]
                        
                        # remove that team form the temp list so we don't pick it again
                        AFC_teams.remove(selected_team_a)
                        # replace the default None value with the weeke_number of their bye
                        bye_list[selected_team_a] = week_num

                        """choose NFC team(s)"""
                        selected_team_n = eligible_teams_nfc[random.randint(0,len(eligible_teams_nfc)-1)]
                        # selected_team_n = eligible_teams_nfc[0]
                        
                        # remove that team form the temp list so we don't pick it again
                        NFC_teams.remove(selected_team_n)
                        # replace the default None value with the weeke_number of their bye
                        bye_list[selected_team_n] = week_num
                        if debug:
                            print(f"{selected_team_a.name = }")
                            print(f"{selected_team_n.name = }")
                    else:
                        # if there are not any eligible teams from one of the conferences, then break and try again
                        keep_searching = True
                        break
            
            if all(value is not None for value in bye_list.values()):
                print("\nSUCCESS!!")
                if debug:
                    print(f"{count = }")
                    print(f"{bye_list = }")
                # All of the teams have been assigned a bye week, so we have succeeded and do not need to try again.
                keep_searching = False
            else:
                keep_searching = True
        
        """ now outside of the while loop """
        print(f"\n{count = }")
        print(f"{bye_list = }")
        for team, byeweek in bye_list.items():
            team.set_bye_week(byeweek)

        return count, cutoff

    def set_schedule_outline(self, debug = False) -> None:
        """ For each division, set aside 6 weeks for divisional games. add those 6 weeks to each teams sheduule outline
            Then add each team's bye week, continue doing until none of the bye weeks overlap with each teams weeks reserved for divisional games 
        """

        ####################################################################################
        # Establish 6 weeks for each division to play against themselves.
        divisions = ["North", "South", "East", "West"]

        # currently excluding week 13 and 14 (to ensure we can always have enough teams eligible for a bye week)
        div_eligible_weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13] + [16,17,18]

        for div in divisions:
            divisional_game_weeks = sorted(np.random.choice(div_eligible_weeks, size = 6, replace = False))
            for team in [t for t in self.allteams if t.division == div]:
                for week_num in divisional_game_weeks:
                    team.schedule_outline[week_num] = "d"
        print("6 divisional weeks assigned")

        ####################################################################################
        # Set up 1 bye week for all teams:
        x, cutoff = self.assign_bye_weeks(debug)
        if x > cutoff:
            print("FAIL")
        print("1 bye week assigned")

        ####################################################################################
        # Reserve one week for across-conference games
        open_weeks = list(range(1,19))
        for team in self.allteams:
            for week in open_weeks:
                # if a team has something scheduled that week, remove it from the options
                if team.schedule_outline[week] is not None:
                    open_weeks.remove(week)
        if debug:
            print(f"{open_weeks = }")

        ####################################################################################
        # Reserve the earliest open week for across-conference games
        xconf_week = min(open_weeks)
        for team in self.allteams:
            team.schedule_outline[xconf_week] = "XC"
        print("1 cross-conference week assigned")

        ####################################################################################
        # designate the remaining weeks as random conference games
        for team in self.allteams:
            for week_num in team.schedule_outline.keys():
                if team.schedule_outline[week_num] == None:
                    team.schedule_outline[week_num] = "conf"
        print("remaining weeks set as non-divisional conference games")

    def add_game_to_teams(self, week: int, home_team: NFLTeam, away_team: NFLTeam) -> None:
        """Accesses the pre-defined object for both teams and adds this matchup to the team objects scheduule
        Args:
            week (int): the week that the two teams are playing each other
            home_team (class <NFLTeam>): the home team 
            away_team (class <NFLTeam>): the away team 
        """
        # check if its a divisional or conference game
        if home_team.conference == home_team.conference:
            home_team.conference_games_played += 1
            away_team.conference_games_played += 1
            # if in conference, check if also in division.
            if home_team.division == home_team.division:
                home_team.division_games_played += 1
                away_team.division_games_played += 1

    def add_game_to_schedule(self, week, hometeam: NFLTeam, awayteam: NFLTeam):
        # self.schedule[week] = {"home": hometeam, "away": awayteam}
        # self.schedule[week] = self.schedule[week] + {"home": home_team, "away": away_team}
        pass

    def set_real_schedule(self, debug = False) -> None:
        for week in self.schedule.keys():
            # need to be sure that every team plays 1 time per week (or has a bye)
            teams_list = self.allteams.copy()

            for team in teams_list:
                if "bye" in team.schedule_outline[week].lower():
                    if debug:
                        print(f"{week = }, bye team = {team.mascot}")




            # print(week)
            # self.schedule[week]


In [342]:
teams = create_teams()
myleague = NFLSchedule(teams)

myleague.set_schedule_outline(debug=False)

6 divisional weeks assigned

SUCCESS!!

count = 57
bye_list = {Patriots A.e: 7, Bills A.e: 7, Dolphins A.e: 4, Jets A.e: 11, Ravens A.n: 8, Steelers A.n: 5, Browns A.n: 9, Bengals A.n: 11, Texans A.s: 9, Colts A.s: 14, Titans A.s: 4, Jaguars A.s: 6, Chiefs A.w: 12, Broncos A.w: 10, Raiders A.w: 13, Chargers A.w: 13, Cowboys N.e: 7, Commanders N.e: 4, Eagles N.e: 4, Giants N.e: 7, Packers N.n: 10, Bears N.n: 9, Vikings N.n: 9, Lions N.n: 8, Buccaneers N.s: 6, Saints N.s: 11, Panthers N.s: 5, Falcons N.s: 11, Seahawks N.w: 13, Rams N.w: 14, Cardinals N.w: 13, 49ers N.w: 12}
1 bye week assigned
1 cross-conference week assigned
remaining weeks set as non-divisional conference games


In [343]:
myleague.set_real_schedule(debug=True)

week = 4, bye team = Dolphins
week = 4, bye team = Titans
week = 4, bye team = Commanders
week = 4, bye team = Eagles
week = 5, bye team = Steelers
week = 5, bye team = Panthers
week = 6, bye team = Jaguars
week = 6, bye team = Buccaneers
week = 7, bye team = Patriots
week = 7, bye team = Bills
week = 7, bye team = Cowboys
week = 7, bye team = Giants
week = 8, bye team = Ravens
week = 8, bye team = Lions
week = 9, bye team = Browns
week = 9, bye team = Texans
week = 9, bye team = Bears
week = 9, bye team = Vikings
week = 10, bye team = Broncos
week = 10, bye team = Packers
week = 11, bye team = Jets
week = 11, bye team = Bengals
week = 11, bye team = Saints
week = 11, bye team = Falcons
week = 12, bye team = Chiefs
week = 12, bye team = 49ers
week = 13, bye team = Raiders
week = 13, bye team = Chargers
week = 13, bye team = Seahawks
week = 13, bye team = Cardinals
week = 14, bye team = Colts
week = 14, bye team = Rams


In [326]:
for team in myleague.allteams:
    print(team.schedule_outline, "- - -",  team)

{1: 'conf', 2: 'd', 3: 'd', 4: 'conf', 5: 'conf', 6: 'conf', 7: 'conf', 8: 'd', 9: 'd', 10: 'conf', 11: '-- BYE --', 12: 'd', 13: 'conf', 14: 'conf', 15: 'XC', 16: 'd', 17: 'conf', 18: 'conf'} - - - Patriots A.e
{1: 'conf', 2: 'd', 3: 'd', 4: 'conf', 5: 'conf', 6: 'conf', 7: '-- BYE --', 8: 'd', 9: 'd', 10: 'conf', 11: 'conf', 12: 'd', 13: 'conf', 14: 'conf', 15: 'XC', 16: 'd', 17: 'conf', 18: 'conf'} - - - Bills A.e
{1: 'conf', 2: 'd', 3: 'd', 4: 'conf', 5: 'conf', 6: 'conf', 7: '-- BYE --', 8: 'd', 9: 'd', 10: 'conf', 11: 'conf', 12: 'd', 13: 'conf', 14: 'conf', 15: 'XC', 16: 'd', 17: 'conf', 18: 'conf'} - - - Dolphins A.e
{1: 'conf', 2: 'd', 3: 'd', 4: 'conf', 5: 'conf', 6: 'conf', 7: 'conf', 8: 'd', 9: 'd', 10: 'conf', 11: '-- BYE --', 12: 'd', 13: 'conf', 14: 'conf', 15: 'XC', 16: 'd', 17: 'conf', 18: 'conf'} - - - Jets A.e
{1: 'd', 2: 'conf', 3: 'd', 4: 'conf', 5: 'conf', 6: 'conf', 7: 'd', 8: 'conf', 9: 'conf', 10: '-- BYE --', 11: 'd', 12: 'conf', 13: 'conf', 14: 'conf', 15: 'X

In [324]:
for team in myleague.allteams:
    print(team.schedule, "- - -",  team)

{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: ' -- BYE --', 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None} - - - Patriots A.e
{1: None, 2: None, 3: None, 4: None, 5: ' -- BYE --', 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None} - - - Bills A.e
{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: ' -- BYE --', 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None} - - - Dolphins A.e
{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: ' -- BYE --', 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None} - - - Jets A.e
{1: None, 2: None, 3: None, 4: ' -- BYE --', 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None} - - - Ravens A.n
{1: None, 2: N

In [ ]:
# # create 100 leagues, see how many times the scheduler fails
# attempts = 1
# results = [NFLSchedule(create_teams(), weeks=18).set_schedule_outline() for _ in range(attempts)] 

# failure_rate = (sum([1 for i in results if i > 10000]) / attempts) * 100.00

# failure_rate

In [115]:
# random.seed(1)

random.randint(1,18)

16

In [ ]:
# ASSIGNING BYE WEEKS
myleague.assign_bye_weeks()


In [ ]:
patriots = myleague.allteams[0]
patriots.schedule_outline

In [ ]:

# print("Assigning bye weeks: \n")
# assign_bye_weeks(nfl_teams)

In [ ]:
nfl_teams

In [ ]:
print(len(bengals.opponents_not_faced))

for team in nfl_teams:
    team.opponents_not_faced = [team for team in nfl_teams]
    # team.opponents_not_faced.remove(team.name)

print(len(bengals.opponents_not_faced))


In [ ]:
# bengals.add_game(2,chargers)
bengals.schedule
# patriots.schedule

In [ ]:
myschedule = NFLSchedule(weeks=18)
full_schedule = {}


for week_num in range(0,19):
# for week_num in range(4,5):
    weekly_matchups = []
    weekly_matchups_names = []

    # list of teams in afc and nfc that are playing a game this week
    afc_eligible_teams = [team for team in AFC if team.bye_week != week_num]
    nfc_eligible_teams = [team for team in NFC if team.bye_week != week_num]

    # shuffle the teams to get randomized matchups
    random.shuffle(afc_eligible_teams)
    random.shuffle(nfc_eligible_teams)

    print(f"\n{week_num = }")
    # print(f"{afc_eligible_teams = }")
    # print(f"{nfc_eligible_teams = }")

    for team in nfl_teams:
        opponent = random.choice(team.opponents_not_faced)
        # team.opponents_not_faced.remove(opponent)
        

    # as if every team plays someone from the other conference
    for afc_team, nfc_team in zip(afc_eligible_teams, nfc_eligible_teams):
        weekly_matchups.append({"home": afc_team, "away": nfc_team})
        weekly_matchups_names.append({"home": afc_team.name, "away": nfc_team.name})
    # print(f"{weekly_matchups_names = }")


    # assign that game to each team's 
    for game in weekly_matchups:
        # print(game)
        home_team = game["home"]
        away_team = game["away"]

        print(away_team, "@", home_team)

        home_team.add_game(week_num, away_team)
        away_team.add_game(week_num, home_team)

        # game["home"].add_game(week_num, game["away"])
        # game["away"].add_game(week_num, game["home"])
        
        # add_game_to_team(week_num, game["home"])
        # add_game_to_team(week_num, game["away"])
        
    full_schedule[f"week_{week_num+1}"] = weekly_matchups
    
# full_schedule_json = json.dumps(full_schedule, indent=4)  # indent for pretty printing
# print(full_schedule_json)
# full_schedule


In [ ]:
# [game.name if type(game) == NFLTeam else game for game in bengals.schedule.values()]
# [game.name if type(game) == NFLTeam else game for game in sf49ers.schedule.values()]

[game.name if type(game) == NFLTeam else game for game in patriots.schedule.values()]
# [game.name if type(game) == NFLTeam else game for game in cowboys.schedule.values()]